# basic-001

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Load datasets
train_df = pd.read_csv("", sep=r"\s+", engine='python')
predict_df = pd.read_csv("last30.csv", sep=r"\s+", engine='python')

In [ ]:
# Use only numerical features
features = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL', 'SPREAD']

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[features])
predict_scaled = scaler.transform(predict_df[features])

In [ ]:
# Create sequences for training
def create_sequences(data, window_size=30):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i][3])  # predicting 'CLOSE' price (index 3)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled)

In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=64)

In [ ]:
# Predict next 100 candles based on the last 30 real ones
predictions = []
input_seq = predict_scaled[-30:]  # start from last 30 rows

for _ in range(100):
    pred_input = np.expand_dims(input_seq, axis=0)
    pred_close = model.predict(pred_input, verbose=0)[0][0]

    # Build the next candle with predicted CLOSE and last known values
    next_candle = input_seq[-1].copy()
    next_candle[3] = pred_close  # replace CLOSE with predicted

    predictions.append(next_candle)
    input_seq = np.vstack([input_seq[1:], next_candle])  # roll the window

In [ ]:
# Inverse transform
all_candles = np.vstack([predict_scaled, predictions])
all_candles = scaler.inverse_transform(all_candles)

In [ ]:
# Plotting
plt.figure(figsize=(14, 6))
real_closes = all_candles[:30, 3]
predicted_closes = all_candles[30:, 3]

plt.plot(range(0, 30), real_closes, label='Real CLOSE', color='blue')
plt.plot(range(30, 130), predicted_closes, label='Predicted CLOSE', color='orange')
plt.axvline(x=29, color='gray', linestyle='--')
plt.title("30 Real + 100 Predicted Candles (CLOSE price)")
plt.xlabel("Candle Index")
plt.ylabel("CLOSE Price")
plt.legend()
plt.grid(True)
plt.show()
